# 1. Import libraries

In [184]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

In [185]:
# !pip install konlpy

In [186]:
# !pip install gensim

In [187]:
#colab에서 실행 중이라면...
# !git clone https://github.com/hukim1112/comment_classifier.git
# import os
# os.chdir('/content/comment_classifier')

In [188]:
import tensorflow as tf
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt


keras = tf.keras
t = Okt()

# 2. fit tokenizer to our datasets

In [189]:
from vectorizer import BaseVectorizer
vectorizer = BaseVectorizer(t.morphs)
tokenizer = BaseVectorizer(t.morphs)

In [190]:
df = pd.read_csv('Entitytest.csv',encoding='CP949')

In [191]:
df[0:1]

,word,entity
0,빨간색,color


In [192]:
tokenizer.fit(df['word'].values)

scanning was done                                        
86 terms are recognized


In [193]:
tokenizer.vocabulary_

{'_PAD_': 0,
 '_UNK_': 1,
 '_STA_': 2,
 '_EOS_': 3,
 '색': 4,
 '보라색': 5,
 '공': 6,
 '위': 7,
 '아래': 8,
 '맨': 9,
 '빨간색': 10,
 '파랑색': 11,
 '빨강': 12,
 '파란색': 13,
 '주황색': 14,
 '노란색': 15,
 '노랑': 16,
 '초록색': 17,
 '녹색': 18,
 '연두색': 19,
 '남색': 20,
 '자주색': 21,
 '하늘색': 22,
 '연': 23,
 '분홍색': 24,
 '청록색': 25,
 '청색': 26,
 '적색': 27,
 '갈색': 28,
 '네이비': 29,
 '자동차': 30,
 '축구공': 31,
 '농구공': 32,
 '피구': 33,
 '럭비공': 34,
 '야구공': 35,
 '테니스공': 36,
 '장난감': 37,
 '의자': 38,
 '책': 39,
 '책상': 40,
 '컵': 41,
 '과자': 42,
 '상자': 43,
 '박스': 44,
 '게임기': 45,
 '컴퓨터': 46,
 '텔레비전': 47,
 '티비': 48,
 '가방': 49,
 '지갑': 50,
 '양말': 51,
 '패딩': 52,
 '외투': 53,
 '티셔츠': 54,
 '셔츠': 55,
 '바지': 56,
 '청바지': 57,
 '핸드폰': 58,
 '휴대폰': 59,
 '키': 60,
 '보드': 61,
 '마우스': 62,
 '모니터': 63,
 '노트북': 64,
 '공책': 65,
 '오른쪽': 66,
 '왼쪽': 67,
 '좌회전': 68,
 '우회': 69,
 '전': 70,
 '좌측': 71,
 '우측': 72,
 '저기': 73,
 '여기': 74,
 '저쪽': 75,
 '동쪽': 76,
 '서쪽': 77,
 '남쪽': 78,
 '북쪽': 79,
 '동': 80,
 '서': 81,
 '남': 82,
 '북': 83,
 '옆': 84,
 '안': 85}

# 3. data preprocessing

In [194]:
label_to_id = {t:i for i,t in enumerate(df.entity.unique())}
id_to_label = {i:t for i,t in enumerate(df.entity.unique())}

In [195]:
print(label_to_id)

{'color': 0, 'thing': 1, 'loc': 2}


In [196]:
# df.intent = df.intent.map(lambda x : label_index[x])
# print(df.head(10))

In [197]:
MAX_LENGTH = 10
def tokenize_and_filter(sentences, labels):
    inputs, outputs = [], []
  
    for sentence, label in zip(sentences, labels):
    # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        
    # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
#             print("input append")
            outputs.append(label_to_id[label])
  
  # pad tokenized sentences
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        inputs, maxlen=MAX_LENGTH, padding='post', 
        value = tokenizer.vocabulary_['_PAD_']) # value = 0
  
    return padded_inputs, outputs

In [198]:
inputs, outputs = tokenize_and_filter(df.word, df.entity)

In [199]:
print('encoded input : ', inputs[0], 'label : ', outputs[0], 'original input sentence : ', tokenizer.decode_from_list(inputs[0]))

encoded input :  [10  0  0  0  0  0  0  0  0  0] label :  0 original input sentence :  ['빨간색', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_']


In [200]:
BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [201]:
dataset

<PrefetchDataset shapes: ((None, 10), (None,)), types: (tf.int32, tf.int32)>

In [202]:
for x, y in dataset.take(1):
    print(x, y)
    print('-----------------------------------------------')
    print(x.shape, y.shape)

tf.Tensor(
[[22  0  0  0  0  0  0  0  0  0]
 [44  0  0  0  0  0  0  0  0  0]
 [58  0  0  0  0  0  0  0  0  0]
 [52  0  0  0  0  0  0  0  0  0]
 [68  0  0  0  0  0  0  0  0  0]
 [ 8  0  0  0  0  0  0  0  0  0]
 [77  0  0  0  0  0  0  0  0  0]
 [62  0  0  0  0  0  0  0  0  0]
 [43  0  0  0  0  0  0  0  0  0]
 [23  5  0  0  0  0  0  0  0  0]
 [51  0  0  0  0  0  0  0  0  0]
 [28  0  0  0  0  0  0  0  0  0]
 [15  0  0  0  0  0  0  0  0  0]
 [16  4  0  0  0  0  0  0  0  0]
 [20  0  0  0  0  0  0  0  0  0]
 [45  0  0  0  0  0  0  0  0  0]], shape=(16, 10), dtype=int32) tf.Tensor([0 1 1 1 2 2 2 1 1 0 1 0 0 0 0 1], shape=(16,), dtype=int32)
-----------------------------------------------
(16, 10) (16,)


# 4. model design

In [203]:
print(len(label_to_id.values()))

3


In [204]:
def get_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.n_vocabs, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_to_id.values()), activation='softmax')
])
    LEARNING_RATE = 0.001
    model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    return model

In [205]:
model = get_model()

In [206]:
# LEARNING_RATE = 0.001

In [207]:
# model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
#               loss=tf.keras.losses.sparse_categorical_crossentropy,
#               metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [209]:
model.fit(dataset, epochs=10)

Epoch 1/10
5/5 [==============================] - 0s 9ms/step - loss: 1.0030 - sparse_categorical_accuracy: 0.4500
Epoch 2/10
5/5 [==============================] - 0s 8ms/step - loss: 0.9841 - sparse_categorical_accuracy: 0.4500
Epoch 3/10
5/5 [==============================] - 0s 8ms/step - loss: 0.9579 - sparse_categorical_accuracy: 0.4500
Epoch 4/10
5/5 [==============================] - 0s 8ms/step - loss: 0.8928 - sparse_categorical_accuracy: 0.4750
Epoch 5/10
5/5 [==============================] - 0s 8ms/step - loss: 0.7851 - sparse_categorical_accuracy: 0.7000
Epoch 6/10
5/5 [==============================] - 0s 8ms/step - loss: 0.6291 - sparse_categorical_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 0s 8ms/step - loss: 0.3752 - sparse_categorical_accuracy: 1.0000
Epoch 8/10
5/5 [==============================] - 0s 8ms/step - loss: 0.1945 - sparse_categorical_accuracy: 1.0000
Epoch 9/10
5/5 [==============================] - 0s 8ms/step - loss: 0.1231 - s

In [210]:
def question_processing(sentences):
    MAX_LENGTH = 10
    inputs = []
    for sentence in sentences:
        # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
        else:
            print('입력이 너무 길어요.')
    # pad tokenized sentences
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, maxlen=MAX_LENGTH, padding='post', 
    value = tokenizer.vocabulary_['_PAD_']) # value = 0
    return padded_inputs

In [211]:
input_sentence = question_processing(['빨간색','휴지','파랑색','비','무지개색','오른쪽','노란색','유니폼','상자','옆','위'
                                     ])

In [212]:
model.predict(input_sentence)

array([[9.4229478e-01, 3.1282444e-04, 5.7392407e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [9.4701344e-01, 2.4948525e-04, 5.2737065e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [9.6862912e-01, 3.1206716e-04, 3.1058801e-02],
       [4.8841119e-02, 8.9883851e-03, 9.4217056e-01],
       [9.4959462e-01, 3.2338305e-04, 5.0081894e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [4.1759838e-05, 9.9968314e-01, 2.7513146e-04],
       [3.4237865e-02, 1.2231455e-02, 9.5353067e-01],
       [2.2944862e-02, 1.5088378e-02, 9.6196675e-01]], dtype=float32)

In [213]:
prediction = np.argmax(model.predict(input_sentence), axis=1)
print(prediction)

[0 1 0 1 0 2 0 1 1 2 2]


In [214]:
for p in prediction:
    print(id_to_label[p])

color
thing
color
thing
color
loc
color
thing
thing
loc
loc


In [224]:
# 모델 저장하기
model.save('entitytestmodel.h5')
# 모델 불러오기
tempmodel = keras.models.load_model('entitytestmodel.h5')
tempmodel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          5504      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 195       
Total params: 80,003
Trainable params: 80,003
Non-trainable params: 0
_________________________________________________________________


In [225]:
tempmodel.predict(input_sentence)

array([[9.4229478e-01, 3.1282444e-04, 5.7392407e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [9.4701344e-01, 2.4948525e-04, 5.2737065e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [9.6862912e-01, 3.1206716e-04, 3.1058801e-02],
       [4.8841119e-02, 8.9883851e-03, 9.4217056e-01],
       [9.4959462e-01, 3.2338305e-04, 5.0081894e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [4.1759838e-05, 9.9968314e-01, 2.7513146e-04],
       [3.4237865e-02, 1.2231455e-02, 9.5353067e-01],
       [2.2944862e-02, 1.5088378e-02, 9.6196675e-01]], dtype=float32)

In [226]:
list(np.argmax(model.predict(input_sentence), axis=1))

[0, 1, 0, 1, 0, 2, 0, 1, 1, 2, 2]

In [227]:
for p in prediction:
    print(id_to_label[p])

color
thing
color
thing
color
loc
color
thing
thing
loc
loc


In [70]:
del model

# 데이터 추가해보기

In [ ]:
names = ['안중근', '이순신', '세종대왕', '김광석', '아이유', '에미넴', '이건희', '고아라', '유재석', '한석희', '최민성']
def question_generator(names):
    question = []
    for name in names:
        s1 = name+'는 어떤 분이야?'
        s2 = name+'은 어떤 사람이니?'
        s3 = name+'이란 사람에 대해 궁금해'
        question = question+[s1, s2, s3]
    return question
question = question_generator(names)

In [ ]:
question

In [ ]:
new_data = {'question' : question, 'intent' : ['인물']*len(question)}
add_df = pd.DataFrame(new_data, columns=('question', 'intent'))

In [ ]:
add_df.head(5)

In [ ]:
print(len(df), len(add_df))

In [ ]:
new_df = pd.concat([df, add_df])
print(len(new_df))

In [ ]:
tokenizer.fit(new_df['question'].values)

In [ ]:
new_inputs, new_outputs = tokenize_and_filter(new_df.question, new_df.intent)

BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((new_inputs, new_outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
new_model = get_model()
LEARNING_RATE = 0.0001
new_model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
new_model.fit(dataset, epochs=10)

In [ ]:
input_sentence = question_processing(['서울 날씨 어때?', 
                                      '나는 전주 날씨 궁금함',
                                      '안중근 의사는 누구야?',
                                      '박소희는 어떤 사람인지 궁금해.',
                                      '명동 맛있는 음식점 있니?'
                                     ])

In [ ]:
new_model.predict(input_sentence)

In [ ]:
prediction = np.argmax(new_model.predict(input_sentence), axis=1)

In [ ]:
for p in prediction:
    print(id_to_label[p])

In [216]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [217]:

def 형태소분석(text):
    형태소 = kkma.pos(text)
    명사 = []
    for i in 형태소:
        if i[1] == 'NNG':
            명사.append(i[0])
        else:
            pass
    return 명사

def entity분석(명사):
    inputdata = question_processing(명사)
    prediction = list(np.argmax(tempmodel.predict(inputdata),axis=1))
    print(명사)
    print(prediction)
    result = {}
    for a,b in zip(명사,prediction):
        if b == 0:
            result[a] = 'color'
        elif b == 1:
            result[a] = 'thing'
        elif b == 2:
            result[a] = 'loc'
    return result  


In [221]:
tempmodel = keras.models.load_model('entitytestmodel.h5')
noun = 형태소분석('빨간색 공책 녹색 상자 안에 넣어')
result = entity분석(noun)

['빨간색', '공책', '녹색', '상자', '안']
[0, 1, 0, 1, 2]


In [222]:
result

{'빨간색': 'color', '공책': 'thing', '녹색': 'color', '상자': 'thing', '안': 'loc'}